In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns
import glob

# --------------- IMPORTING THE DATA --------------- #

# Evacuation Data
evacdata = pd.read_csv('./outputs/gy-evac res-table_20runs_prob0p3.csv', skiprows=6).sort_values(by=['tick-time-in-mins', 'ticks']).reset_index(drop=True)
# renaming the column to something that isn't cursed
evacdata.rename(columns={'[run number]': 'run_number'}, inplace=True)
evacdata.columns = evacdata.columns.str.replace('-', '_')

# Journey Time Data
jm_files = glob.glob('./outputs/jm*res.csv')
journey_metrics_list = [pd.read_csv(file) for file in jm_files]
# Extract tick number from file names and add as a column to each dataframe
for i, file in enumerate(jm_files):
    tick_number = float(file.split('_')[4][4:])
    journey_metrics_list[i]['tick_number'] =tick_number
# Concatenate the dataframes
journey_metrics = pd.concat(journey_metrics_list, ignore_index=True).sort_values(by=['tick_number', 'start_tick'])

# calculate normalised journey time
journey_metrics['normalised_journey_time'] = journey_metrics['actual_time'] / journey_metrics['ideal_time']

# calculate the mean normalised journey time every 30 minutes for each run
# Bin the start_tick at intervals of 120 (corresponding to 30 minutes)
journey_metrics['start_tick_bin'] = pd.cut(journey_metrics['start_tick'], bins=np.arange(0, journey_metrics['start_tick'].max() + 120, 120))
# Calculate the mean normalised journey time for each bin
mean_normalised_journey_time_per_bin = journey_metrics.groupby('start_tick_bin')['normalised_journey_time'].mean().reset_index()
# Calculate the mean normalised journey time for each bin for each run
mean_normalised_journey_time_per_bin_per_run = journey_metrics.groupby(['tick_number', 'start_tick_bin'])['normalised_journey_time'].mean().reset_index()

In [8]:
evacdata.head()

,run_number,over_break_p,terminate_evac_distance_km,initial_people,max_walking_distance_km,warning_interval_time_mins,evacuation_probability,tick_time_in_mins,[step],ticks,get_no_active_cars,get_no_walking,get_no_evacuating,get_no_evacuated,get_no_in_cars,get_avg_no_people_per_car
0,1,0.06,0.3,15000,0.3,30,0.3,0.05,0,0,0,0,0,0,0,0.000000
1,1,0.06,0.3,15000,0.3,30,0.3,0.05,1,1,1214,0,3446,21,3446,2.838550
2,1,0.06,0.3,15000,0.3,30,0.3,0.05,2,2,1213,0,3446,21,3446,2.840890
3,1,0.06,0.3,15000,0.3,30,0.3,0.05,3,3,1207,0,3433,34,3433,2.844242
4,1,0.06,0.3,15000,0.3,30,0.3,0.05,4,4,1205,0,3429,38,3429,2.845643
